In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !git clone https://github.com/MJC598/Neuron_Burst_Analysis.git

In [ ]:
%matplotlib widget
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.io
import random
import time
import pandas as pds
from sklearn.metrics import r2_score

from abc import abstractmethod

torch.manual_seed(0)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
    def generate(self, x):
        raise NotImplementedError
        
    """
    KL Divergence and Reconstruction derived from:
    https://towardsdatascience.com/variational-autoencoder-demystified-with-pytorch-implementation-3a06bee395ed
    """
    def kl_div(self, sample, mean, std):
        p = torch.distributions.Normal(torch.zeros_like(mean), torch.ones_like(std)).to(self.device)
        q = torch.distributions.Normal(mean, std).to(self.device)

        log_qzx = q.log_prob(sample)
        log_pz = p.log_prob(sample)

        kl = (log_qzx - log_pz)
        return kl.sum(-1)
    
    def reconstruction(self, x_hat, x, dims=(1,2,3)):
        scale = torch.exp(nn.Parameter(torch.Tensor([0.0])).to(self.device)).to(self.device)
        dist = torch.distributions.Normal(x_hat, scale).to(self.device)
        return dist.log_prob(x).sum(dim=dims)
        
    def ELBOLoss(self, x, sample, mean, std, x_hat):
        return (self.kl_div(sample, mean, std) - self.reconstruction(x_hat, x)).mean()
        
    @abstractmethod
    def forward(self, x):
        pass
    

In [ ]:
class LSTM_VAE(VAE):
    def __init__(self):
        super(LSTM_VAE, self).__init__()
        self.encoder = nn.Sequential(
                        nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout),
                        nn.Linear(in_features=in_features, out_features=out_features), #needs to encode from LSTM Dims to Latent Dims
                        nn.ReLU()
                    )
        
    def forward(self, x):
        return x

In [ ]:
class GRU_VAE(VAE):
    def __init__(self):
        super(GRU_VAE, self).__init__()
    
    def forward(self, x):
        return x